<a href="https://colab.research.google.com/github/MACHS92/03MIAR_10_A_2023-24_Algoritmos_de_Optimizacion/blob/main/Manuel_Chiriboga_Trabajo_Pr%C3%A1ctico_Algoritmos(V2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Manuel Alejandro Chiriboga Salazar  <br>
Url: https://github.com/MACHS92/03MIAR_10_A_2023-24_Algoritmos_de_Optimizacion<br>
Google Colab: https://colab.research.google.com/drive/1uhCtXpNBSYqvIh4YpLesQ3waDD2hzYCs#scrollTo=hVbXYX-RfPWh <br>
Problema:

>3. Configuración de Tribunales

 Se precisa configurar tribunales de evaluación para un grupo de 15 alumnos que desean presentar su Trabajo Fin de Máster (TFM).

 Cada tribunal está compuesto por tres profesores, cada uno desempeñando uno de lossiguientes roles: Presidente, Secretario o Vocal.

Los profesores han indicado su disponibilidad horaria para participar en los tribunales de 15h a 21h durante la semana del 15 al 19 de abril:

Número de profesores : 10

Número de tribunales : 15

Disponibilidad/Roles : https://bit.ly/41QWk8o
- 1 indica que profesor tiene disponibilidad
- 0 en caso contrario










                                        

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

Para resolver este problema de optimización debemos empezar por reconocer cual es la variable para optimizar. En el mismo en este caso sería el tiempo de cada profesor en los tribunales de TFM, evitando solapamientos , o incurriendo en choques de horarios o que algunos profesores tengan mas o menos horas de TFM.

##¿Como represento el espacio de soluciones?

Par modelar el espacio de solución tenemos  una matriz donde las columna representan los tribunales y las filas representan los profesores. Los valores de la matriz serían binarios, indicando si un profesor está asignado a un tribunal o no.



##¿Cual es la función objetivo?

 La función objetivo seria la minimización del solapamiento de horarios entre los tribunales. Por ejemplo, podríamos minimizar la cantidad total de horas en las que dos tribunales tienen al menos un profesor en común o también disminuir la repetición de profesores en un tribunal para que sea equitativa la distribución.  

##¿Como implemento las restricciones?

1.-Los profesores solo pueden ser asignados a tribunales durante sus horas de disponibilidad

2.-Cada tribunal debe tener exactamente tres profesores asignados.

3.-Cada profesor puede participar en un máximo de un tribunal a la vez.


In [ ]:
#Respuesta



#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

Como se observa en el programa anteriormente aplicado podemos notar que esta relaciona directamente  a los tres roles de los profesores y sus impedimentos horarios además impedimentos de los mismos roles . Por lo mismo Por lo tanto, la complejidad del algoritmo es exponencial en términos del número de tribunales, lo que lo hace computacionalmente costoso.

En cuanto al espacio de soluciones, cada tribunal puede tener diferentes combinaciones de profesores en cada uno de los tres roles. Dado que hay 10 profesores y 3 roles, el espacio de soluciones total para un tribunal es 103=1000. Dado que hay 15 tribunales en total, el espacio de soluciones completo es 100015, lo que representa una cantidad extremadamente grande de posibles configuraciones.

#Diseño
- ¿Que técnica utilizo? ¿Por qué?

En este caso después de analizar y probar un par de algoritmos concretamente genéticos con  la  librería deap de python  y backtracking con poda llegamos a la conclusión que la manera mas optima de conseguir una solución es backtracking con poda

In [24]:
#Importaciones
import os
import csv
import math
import random
import pandas as pd


In [ ]:

#Cargar  Archivos
# Función para cargar la disponibilidad y los roles de los profesores desde archivos CSV
def cargar_datos(archivo_disponibilidad, archivo_roles):
    disponibilidad = []
    roles = []

    # Cargar disponibilidad
    with open(archivo_disponibilidad, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            disponibilidad.append([int(val) for val in row])

    # Cargar roles
    with open(archivo_roles, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            roles.append([int(val) for val in row])

    return disponibilidad, roles

# Función de verificación de solución parcial
def es_solucion_parcial_valida(solucion_parcial):
    # Implementa la lógica para verificar la validez de la solución parcial
    # Verificar la disponibilidad y los roles de los profesores asignados a cada tribunal
    pass

# Función de backtracking
def backtracking(tribunales, solucion_parcial, disponibilidad, roles):
    # Caso base: si se han asignado profesores a todos los tribunales
    if len(solucion_parcial) == tribunales:
        if es_solucion_parcial_valida(solucion_parcial):
            return solucion_parcial
        else:
            return None

    # Seleccionar el próximo tribunal
    proximo_tribunal = len(solucion_parcial)

    # Iterar sobre los profesores disponibles para el próximo tribunal
    for profesor_idx, disponibilidad_profesor in enumerate(disponibilidad):
        if disponibilidad_profesor[proximo_tribunal] == 1:
            for rol_idx, rol_disponible in enumerate(roles[profesor_idx]):
                if rol_disponible == 1:
                    # Asignar el profesor al tribunal con el rol correspondiente
                    solucion_parcial.append((profesor_idx, rol_idx))

                    # Llamar recursivamente al algoritmo de backtracking para el próximo tribunal
                    resultado = backtracking(tribunales, solucion_parcial, disponibilidad, roles)
                    if resultado:
                        return resultado

                    # Retroceder si no se encontró una solución válida
                    solucion_parcial.pop()

    # Si no se encontró una solución válida
    return None

# Función principal
def configurar_tribunales(archivo_disponibilidad, archivo_roles, num_tribunales):
    # Cargar datos desde archivos CSV
    disponibilidad, roles = cargar_datos(archivo_disponibilidad, archivo_roles)

    # Ejecutar el algoritmo de backtracking
    solucion = backtracking(num_tribunales, [], disponibilidad, roles)
    # Devolver la solución encontrada
    return solucion

# Ejemplo de uso
archivo_disponibilidad = 'tribunal_horarios.csv'
archivo_roles = 'tribunal_roles.csv'
num_tribunales = 15
solucion = configurar_tribunales(archivo_disponibilidad, archivo_roles, num_tribunales)

# Llenar el DataFrame con los tribunales configurados
for idx, tribunal in enumerate(tribunales_configurados):
    for j, profesor in enumerate(tribunal):
        for i, disponibilidad_profesor in enumerate(profesor):
            if disponibilidad_profesor == 1:
                tribunales_df.iloc[idx, j] = i + 1  # Agregar 1 para que los índices comiencen en 1

# Guardar el DataFrame en un archivo CSV
tribunales_df.to_csv("tribunales_configurados.csv", index=False)
